In [1]:
import pandas as pd
import numpy as np
import torch
from sklearn import preprocessing
from dgl.data import DGLDataset
import dgl
import time
import networkx as nx
import category_encoders as ce
import torch.nn as nn
import torch.nn.functional as F
import dgl.function as fn
import torch
import tqdm
import math

from typing import *
from sklearn.preprocessing import StandardScaler, Normalizer
import socket
import struct
import random
from sklearn.model_selection import train_test_split

/home/kienho/miniforge3/envs/nlp-env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
file_name = "NF-CSE-CIC-IDS2018-v2.parquet"
data = pd.read_parquet(file_name)

In [3]:
##ADDED
file_name_test = "NF-CSE-CIC-IDS2018-v3.parquet"
test_data = pd.read_parquet(file_name_test)
##

In [4]:
data.Label.value_counts()

Label
0    16635567
1     2258141
Name: count, dtype: int64

In [5]:
##ADDED
test_data.Label.value_counts()
##

Label
0    17514626
1     2600903
Name: count, dtype: int64

In [6]:
data.rename(columns=lambda x: x.strip(), inplace=True)
data['IPV4_SRC_ADDR'] = data["IPV4_SRC_ADDR"].apply(str)
data['L4_SRC_PORT'] = data["L4_SRC_PORT"].apply(str)
data['IPV4_DST_ADDR'] = data["IPV4_DST_ADDR"].apply(str)
data['L4_DST_PORT'] = data["L4_DST_PORT"].apply(str)

In [7]:
##ADDED
test_data.rename(columns=lambda x: x.strip(), inplace=True)
test_data['IPV4_SRC_ADDR'] = test_data["IPV4_SRC_ADDR"].apply(str)
test_data['L4_SRC_PORT'] = test_data["L4_SRC_PORT"].apply(str)
test_data['IPV4_DST_ADDR'] = test_data["IPV4_DST_ADDR"].apply(str)
test_data['L4_DST_PORT'] = test_data["L4_DST_PORT"].apply(str)
##

In [8]:
data.drop(columns=["L4_SRC_PORT", "L4_DST_PORT"], inplace=True)

In [9]:
##ADDED
test_data.drop(columns=["L4_SRC_PORT", "L4_DST_PORT"], inplace=True)
##

In [10]:
data.Attack.unique()

array(['SSH-Bruteforce', 'Benign', 'DDoS attacks-LOIC-HTTP',
       'DDOS attack-HOIC', 'DoS attacks-Slowloris', 'DoS attacks-Hulk',
       'FTP-BruteForce', 'Infilteration', 'Bot', 'DoS attacks-GoldenEye',
       'Brute Force -Web', 'DoS attacks-SlowHTTPTest', 'SQL Injection',
       'DDOS attack-LOIC-UDP', 'Brute Force -XSS'], dtype=object)

In [11]:
##ADDED
test_data.Attack.unique()
##

array(['Benign', 'FTP-BruteForce', 'SSH-Bruteforce',
       'DoS_attacks-GoldenEye', 'DoS_attacks-Slowloris',
       'DoS_attacks-SlowHTTPTest', 'DoS_attacks-Hulk',
       'DDoS_attacks-LOIC-HTTP', 'DDOS_attack-LOIC-UDP',
       'DDOS_attack-HOIC', 'Brute_Force_-Web', 'Brute_Force_-XSS',
       'SQL_Injection', 'Infilteration', 'Bot'], dtype=object)

In [12]:
##ADDED
data["Attack"] = data["Attack"].str.replace(" ", "_")
##

In [13]:
data = data.groupby(by='Attack').sample(frac=0.02, random_state=13)

In [14]:
##ADDED
test_data = test_data.groupby(by='Attack').sample(frac=0.02, random_state=13)
##

In [15]:
data.groupby(by="Attack").count()

,IPV4_SRC_ADDR,IPV4_DST_ADDR,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,...,NUM_PKTS_1024_TO_1514_BYTES,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,Label
Attack,,,,,,,,,,,,,,,,,,,,,
Benign,332711,332711,332711,332711,332711,332711,332711,332711,332711,332711,...,332711,332711,332711,332711,332711,332711,332711,332711,332711,332711
Bot,2862,2862,2862,2862,2862,2862,2862,2862,2862,2862,...,2862,2862,2862,2862,2862,2862,2862,2862,2862,2862
Brute_Force_-Web,43,43,43,43,43,43,43,43,43,43,...,43,43,43,43,43,43,43,43,43,43
Brute_Force_-XSS,19,19,19,19,19,19,19,19,19,19,...,19,19,19,19,19,19,19,19,19,19
DDOS_attack-HOIC,21617,21617,21617,21617,21617,21617,21617,21617,21617,21617,...,21617,21617,21617,21617,21617,21617,21617,21617,21617,21617
DDOS_attack-LOIC-UDP,42,42,42,42,42,42,42,42,42,42,...,42,42,42,42,42,42,42,42,42,42
DDoS_attacks-LOIC-HTTP,6146,6146,6146,6146,6146,6146,6146,6146,6146,6146,...,6146,6146,6146,6146,6146,6146,6146,6146,6146,6146
DoS_attacks-GoldenEye,554,554,554,554,554,554,554,554,554,554,...,554,554,554,554,554,554,554,554,554,554
DoS_attacks-Hulk,8653,8653,8653,8653,8653,8653,8653,8653,8653,8653,...,8653,8653,8653,8653,8653,8653,8653,8653,8653,8653


In [16]:
##ADDED
test_data.groupby(by="Attack").count()
##

,FLOW_START_MILLISECONDS,FLOW_END_MILLISECONDS,IPV4_SRC_ADDR,IPV4_DST_ADDR,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,...,FTP_COMMAND_RET_CODE,SRC_TO_DST_IAT_MIN,SRC_TO_DST_IAT_MAX,SRC_TO_DST_IAT_AVG,SRC_TO_DST_IAT_STDDEV,DST_TO_SRC_IAT_MIN,DST_TO_SRC_IAT_MAX,DST_TO_SRC_IAT_AVG,DST_TO_SRC_IAT_STDDEV,Label
Attack,,,,,,,,,,,,,,,,,,,,,
Benign,350293,350293,350293,350293,350293,350293,350293,350293,350293,350293,...,350293,350293,350293,350293,350293,350293,350293,350293,350293,350293
Bot,4154,4154,4154,4154,4154,4154,4154,4154,4154,4154,...,4154,4154,4154,4154,4154,4154,4154,4154,4154,4154
Brute_Force_-Web,32,32,32,32,32,32,32,32,32,32,...,32,32,32,32,32,32,32,32,32,32
Brute_Force_-XSS,10,10,10,10,10,10,10,10,10,10,...,10,10,10,10,10,10,10,10,10,10
DDOS_attack-HOIC,20646,20646,20646,20646,20646,20646,20646,20646,20646,20646,...,20646,20646,20646,20646,20646,20646,20646,20646,20646,20646
DDOS_attack-LOIC-UDP,69,69,69,69,69,69,69,69,69,69,...,69,69,69,69,69,69,69,69,69,69
DDoS_attacks-LOIC-HTTP,5772,5772,5772,5772,5772,5772,5772,5772,5772,5772,...,5772,5772,5772,5772,5772,5772,5772,5772,5772,5772
DoS_attacks-GoldenEye,1226,1226,1226,1226,1226,1226,1226,1226,1226,1226,...,1226,1226,1226,1226,1226,1226,1226,1226,1226,1226
DoS_attacks-Hulk,2002,2002,2002,2002,2002,2002,2002,2002,2002,2002,...,2002,2002,2002,2002,2002,2002,2002,2002,2002,2002


In [17]:
# X = data.drop(columns=["Attack", "Label"])
# y = data[["Attack", "Label"]]

# X_train, X_test, y_train, y_test = train_test_split(
#         X, y, test_size=0.3, random_state=13, stratify=y)
# ADDED
X = test_data.drop(columns=["Attack", "Label", "FLOW_START_MILLISECONDS", "FLOW_END_MILLISECONDS",
                                 "SRC_TO_DST_IAT_MIN", "SRC_TO_DST_IAT_MAX", "SRC_TO_DST_IAT_AVG",
                                 "SRC_TO_DST_IAT_STDDEV", "DST_TO_SRC_IAT_MIN", "DST_TO_SRC_IAT_MAX",
                                 "DST_TO_SRC_IAT_AVG", "DST_TO_SRC_IAT_STDDEV"])
y = test_data[["Attack", "Label"]]
X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=13, stratify=y)
##

In [18]:
encoder = ce.TargetEncoder(cols=['TCP_FLAGS','L7_PROTO','PROTOCOL',
                                  'CLIENT_TCP_FLAGS','SERVER_TCP_FLAGS','ICMP_TYPE',
                                  'ICMP_IPV4_TYPE','DNS_QUERY_ID','DNS_QUERY_TYPE',
                                  'FTP_COMMAND_RET_CODE'])
encoder.fit(X_train, y_train.Label)

# Transform on training set
X_train = encoder.transform(X_train)

# Transform on testing set
X_test = encoder.transform(X_test)

In [19]:
X_train.replace([np.inf, -np.inf], np.nan, inplace=True)
X_test.replace([np.inf, -np.inf], np.nan, inplace=True)
X_train.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)

In [20]:
# (Modified)
scaler = Normalizer()
cols_to_norm = list(set(list(X_train.iloc[:, 2:].columns))) # Ignore first two as the represents IP addresses
scaler.fit(X_train[cols_to_norm])

# Transform on training set
X_train[cols_to_norm] = scaler.transform(X_train[cols_to_norm])
X_train['h'] = X_train.iloc[:, 2:].values.tolist()
X_train['id'] = X_train.index

# Transform on testing set
X_test[cols_to_norm] = scaler.transform(X_test[cols_to_norm])
X_test['h'] = X_test.iloc[:, 2:].values.tolist()
X_test['id'] = X_test.index

train = pd.concat([X_train, y_train], axis=1)
test = pd.concat([X_test, y_test], axis=1)

In [21]:
X_train.head()

,IPV4_SRC_ADDR,IPV4_DST_ADDR,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,...,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,h,id
18744204,172.31.69.14,18.219.211.138,4.740102e-07,1.323114e-06,0.001264,0.000012,0.000801,0.000012,1.045428e-06,1.016509e-06,...,0.019249,0.063169,3.165538e-07,3.165538e-07,4.626324e-07,4.626324e-07,0.000000,3.038169e-07,"[4.7401022514464873e-07, 1.3231141133088036e-0...",18744204
16092777,172.31.64.33,169.254.169.254,8.691857e-08,2.426175e-07,0.000155,0.000002,0.000196,0.000002,1.177649e-07,1.037055e-07,...,0.003530,0.007722,5.804601e-08,5.804601e-08,8.483223e-08,8.483223e-08,0.000000,5.571047e-08,"[8.691857374655049e-08, 2.426175334037123e-07,...",16092777
8585764,18.219.9.1,172.31.69.28,3.946205e-07,1.101512e-06,0.001047,0.000010,0.002244,0.000010,8.703343e-07,8.462588e-07,...,0.128201,0.052589,2.635357e-07,2.635357e-07,3.851483e-07,3.851483e-07,0.000000,2.529321e-07,"[3.946205413164243e-07, 1.1015121191910754e-06...",8585764
9151507,18.218.229.235,172.31.69.28,2.759324e-07,7.702155e-07,0.000756,0.000007,0.001569,0.000007,6.085679e-07,5.917335e-07,...,0.089643,0.036772,1.842733e-07,1.842733e-07,2.693090e-07,2.693090e-07,0.000000,1.768589e-07,"[2.759323680481234e-07, 7.702154745117165e-07,...",9151507
14212479,172.31.65.38,172.31.0.2,8.558578e-07,8.375243e-07,0.003386,0.000059,0.010988,0.000059,8.503230e-07,8.503230e-07,...,0.000000,0.000000,8.001776e-06,8.001776e-06,6.389747e-06,8.525785e-07,0.003564,7.679815e-06,"[8.558577987023315e-07, 8.375242601934382e-07,...",14212479


In [22]:
lab_enc = preprocessing.LabelEncoder()
lab_enc.fit(data["Attack"])

# Transform on training set
train["Attack"] = lab_enc.transform(train["Attack"])

# Transform on testing set
test["Attack"] = lab_enc.transform(test["Attack"])

In [23]:
# Training graph (Modified)

train['id'] = train.index

train_g = nx.from_pandas_edgelist(train, "IPV4_SRC_ADDR", "IPV4_DST_ADDR",
           ["h", "Label", "Attack", "id"], create_using=nx.MultiGraph())
train_g = train_g.to_directed()
train_g = dgl.from_networkx(train_g, edge_attrs=['h', 'Attack', 'Label', "id"])
nfeat_weight = torch.ones([train_g.number_of_nodes(),
train_g.edata['h'].shape[1]])
train_g.ndata['h'] = nfeat_weight

test['id'] = test.index

# Testing graph
test_g = nx.from_pandas_edgelist(test, "IPV4_SRC_ADDR", "IPV4_DST_ADDR",
            ["h", "Label", "Attack", "id"], create_using=nx.MultiGraph())

test_g = test_g.to_directed()
test_g = dgl.from_networkx(test_g, edge_attrs=['h', 'Attack', 'Label', "id"])
nfeat_weight = torch.ones([test_g.number_of_nodes(),
test_g.edata['h'].shape[1]])
test_g.ndata['h'] = nfeat_weight

In [24]:
import torch.nn as nn
import torch.nn.functional as F
import dgl.function as fn
import tqdm
import gc

class SAGELayer(nn.Module):
    def __init__(self, ndim_in, edims, ndim_out, activation):
      super(SAGELayer, self).__init__()
      self.W_apply = nn.Linear(ndim_in + edims , ndim_out)
      self.activation = F.relu
      self.W_edge = nn.Linear(128 * 2, 256)
      self.reset_parameters()

    def reset_parameters(self):
      gain = nn.init.calculate_gain('relu')
      nn.init.xavier_uniform_(self.W_apply.weight, gain=gain)

    def message_func(self, edges):
      return {'m':  edges.data['h']}

    def forward(self, g_dgl, nfeats, efeats):
      with g_dgl.local_scope():
        g = g_dgl
        g.ndata['h'] = nfeats
        g.edata['h'] = efeats
        g.update_all(self.message_func, fn.mean('m', 'h_neigh'))
        g.ndata['h'] = F.relu(self.W_apply(torch.cat([g.ndata['h'], g.ndata['h_neigh']], 2)))

        # Compute edge embeddings
        u, v = g.edges()
        edge = self.W_edge(torch.cat((g.srcdata['h'][u], g.dstdata['h'][v]), 2))
        return g.ndata['h'], edge

In [25]:
class SAGE(nn.Module):
    def __init__(self, ndim_in, ndim_out, edim,  activation):
      super(SAGE, self).__init__()
      self.layers = nn.ModuleList()
      self.layers.append(SAGELayer(ndim_in, edim, 128, F.relu))

    def forward(self, g, nfeats, efeats, corrupt=False):
      if corrupt:
        e_perm = torch.randperm(g.number_of_edges())
        #n_perm = torch.randperm(g.number_of_nodes())
        efeats = efeats[e_perm]
        #nfeats = nfeats[n_perm]
      for i, layer in enumerate(self.layers):
        #nfeats = layer(g, nfeats, efeats)
        nfeats, e_feats = layer(g, nfeats, efeats)
      #return nfeats.sum(1)
      return nfeats.sum(1), e_feats.sum(1)

In [26]:
class Discriminator(nn.Module):
    def __init__(self, n_hidden):
      super(Discriminator, self).__init__()
      self.weight = nn.Parameter(torch.Tensor(n_hidden, n_hidden))
      self.reset_parameters()

    def uniform(self, size, tensor):
      bound = 1.0 / math.sqrt(size)
      if tensor is not None:
        tensor.data.uniform_(-bound, bound)

    def reset_parameters(self):
      size = self.weight.size(0)
      self.uniform(size, self.weight)

    def forward(self, features, summary):
      features = torch.matmul(features, torch.matmul(self.weight, summary))
      return features

In [27]:
class DGI(nn.Module):
    def __init__(self, ndim_in, ndim_out, edim, activation):
      super(DGI, self).__init__()
      self.encoder = SAGE(ndim_in, ndim_out, edim,  F.relu)
      #self.discriminator = Discriminator(128)
      self.discriminator = Discriminator(256)
      self.loss = nn.BCEWithLogitsLoss()

    def forward(self, g, n_features, e_features):
      positive = self.encoder(g, n_features, e_features, corrupt=False)
      negative = self.encoder(g, n_features, e_features, corrupt=True)
      self.loss = nn.BCEWithLogitsLoss()

    def forward(self, g, n_features, e_features):
      positive = self.encoder(g, n_features, e_features, corrupt=False)
      negative = self.encoder(g, n_features, e_features, corrupt=True)

      positive = positive[1]
      negative = negative[1]

      summary = torch.sigmoid(positive.mean(dim=0))

      positive = self.discriminator(positive, summary)
      negative = self.discriminator(negative, summary)

      l1 = self.loss(positive, torch.ones_like(positive))
      l2 = self.loss(negative, torch.zeros_like(negative))

      return l1 + l2

In [28]:
ndim_in = train_g.ndata['h'].shape[1]
hidden_features = 128
ndim_out = 128
num_layers = 1
edim = train_g.edata['h'].shape[1]
learning_rate = 1e-3
epochs = 4000

In [29]:
dgi = DGI(ndim_in,
    ndim_out,
    edim,
    F.relu)

dgi = dgi.to('cuda')

dgi_optimizer = torch.optim.Adam(dgi.parameters(),
                lr=1e-3,
                weight_decay=0.)

In [30]:
# Format node and edge features for E-GraphSAGE
train_g.ndata['h'] = torch.reshape(train_g.ndata['h'],
                                   (train_g.ndata['h'].shape[0], 1,
                                    train_g.ndata['h'].shape[1]))

train_g.edata['h'] = torch.reshape(train_g.edata['h'],
                                   (train_g.edata['h'].shape[0], 1,
                                    train_g.edata['h'].shape[1]))

In [31]:
# Convert to GPU
train_g = train_g.to('cuda')

In [32]:
dgi.load_state_dict(torch.load('best_dgi_CSE_v2.pkl'))

/tmp/ipykernel_46077/1688994637.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  dgi.load_state_dict(torch.load('best_dgi_CSE_v2.pkl'))


<All keys matched successfully>

In [33]:
training_emb = dgi.encoder(train_g, train_g.ndata['h'], train_g.edata['h'])[1]
training_emb = training_emb.detach().cpu().numpy()

In [34]:
test_g.ndata['h'] = torch.reshape(test_g.ndata['h'],
                                   (test_g.ndata['h'].shape[0], 1,
                                    test_g.ndata['h'].shape[1]))



test_g.edata['h'] = torch.reshape(test_g.edata['h'],
                                   (test_g.edata['h'].shape[0], 1,
                                    test_g.edata['h'].shape[1]))

In [35]:
# Convert to GPU
test_g = test_g.to('cuda')

In [36]:
testing_emb = dgi.encoder(test_g, test_g.ndata['h'], test_g.edata['h'])[1]
testing_emb = testing_emb.detach().cpu().numpy()

In [37]:
# Multimodal (Fusion) Learning

df_train = pd.DataFrame(training_emb,)
# map the id to the original data
df_train['id'] = train_g.edata['id'].detach().cpu().numpy()


df_raw_train = pd.DataFrame(X_train.drop(columns=["IPV4_SRC_ADDR", "IPV4_DST_ADDR", "h"]))
df_fuse_train = pd.merge(df_train, df_raw_train, on='id', how='left')
df_fuse_train = df_fuse_train.drop(columns=["id"])
df_fuse_train["Attacks"] = train_g.edata['Attack'].detach().cpu().numpy()
df_fuse_train["Label"] = train_g.edata['Label'].detach().cpu().numpy()

df_test = pd.DataFrame(testing_emb,)
# map the id to the original data
df_test['id'] = test_g.edata['id'].detach().cpu().numpy()

df_raw_test = pd.DataFrame(X_test.drop(columns=["IPV4_SRC_ADDR", "IPV4_DST_ADDR", "h"]))
df_raw_test = pd.merge(df_test, df_raw_test, on='id', how='left')
df_fuse_test = df_raw_test.drop(columns=["id"])
df_fuse_test["Attacks"] = test_g.edata['Attack'].detach().cpu().numpy()
df_fuse_test["Label"] = test_g.edata['Label'].detach().cpu().numpy()

# Embeddings CBLOF  Embeddings

In [38]:
import torch
import dgl
import numpy as np
import pandas as pd
import torch.optim as optim
import time
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
from sklearn.ensemble import IsolationForest
import gc

from tqdm import tqdm
import itertools

In [39]:
benign_fuse_train_samples = df_fuse_train[df_fuse_train.Label == 0].drop(columns=["Label", "Attacks"])
normal_fuse_train_samples = df_fuse_train.drop(columns=["Label", "Attacks"])

fuse_train_labels = df_fuse_train["Label"]
fuse_test_labels = df_fuse_test["Label"]

fuse_test_samples = df_fuse_test.drop(columns=["Label", "Attacks"])

In [40]:
fuse_test_samples

,0,1,2,3,4,5,6,7,8,9,...,NUM_PKTS_512_TO_1024_BYTES,NUM_PKTS_1024_TO_1514_BYTES,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE
0,0.069223,-0.008266,0.007175,-0.030928,0.084612,-0.093248,-0.031110,0.068222,0.026397,0.083662,...,0.000010,0.000010,0.039284,0.205148,6.460231e-07,6.460231e-07,9.441403e-07,9.441403e-07,0.000000,6.200296e-07
1,0.059322,-0.009390,0.000261,0.003226,0.040232,-0.060536,-0.024252,0.075962,0.016806,0.078841,...,0.000000,0.000000,0.038075,0.026982,2.019403e-05,2.019403e-05,2.951287e-05,2.951287e-05,0.000000,1.938150e-05
2,0.145025,0.030021,-0.004276,-0.172147,0.085786,-0.229110,-0.086503,0.012325,0.247465,0.144891,...,0.000015,0.000015,0.124720,0.974379,2.051016e-06,2.051016e-06,2.997489e-06,2.997489e-06,0.000000,1.968491e-06
3,0.145025,0.030021,-0.004276,-0.172147,0.085786,-0.229110,-0.086503,0.012325,0.247465,0.144891,...,0.000011,0.000011,0.702095,0.699472,1.472351e-06,1.472351e-06,2.151790e-06,2.151790e-06,0.000000,1.413109e-06
4,0.070924,0.005745,0.014359,0.019271,-0.001987,-0.030582,-0.006403,0.066014,0.013392,0.058639,...,0.000000,0.000000,0.000000,0.000000,1.919235e-07,1.919235e-07,1.580721e-07,2.044919e-08,0.000085,1.842012e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240747,0.099718,0.027346,0.001190,-0.029865,0.050578,-0.093465,0.019443,0.081388,0.058837,0.090973,...,0.000000,0.000123,0.502245,0.502245,8.259376e-06,8.259376e-06,1.207079e-05,1.207079e-05,0.000000,7.927051e-06
240748,0.058736,0.009330,0.014312,0.005292,0.007657,-0.059158,-0.011096,0.047464,0.035043,0.061368,...,0.000000,0.000000,0.146438,0.000000,3.785920e-07,3.785920e-07,5.532990e-07,5.532990e-07,0.000000,3.633589e-07
240749,0.043863,-0.008849,-0.020216,-0.012184,0.024724,-0.044794,0.002264,0.046857,0.041136,0.072255,...,0.000000,0.000000,0.583955,0.291249,4.789559e-06,4.789559e-06,6.999774e-06,6.999774e-06,0.000000,4.596846e-06
240750,0.130547,0.055592,-0.042487,-0.134789,0.009127,-0.168493,0.029759,0.077130,0.110250,0.126917,...,0.000014,0.000014,0.447120,0.873094,1.837817e-06,1.837817e-06,2.685905e-06,2.685905e-06,0.000000,1.763870e-06


In [41]:
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]

In [42]:
from pyod.models.cblof import CBLOF
n_est = [5,6,7,9,10] # cant be lower than 5 or 4
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    clf_if = CBLOF(n_clusters=n_est, contamination=con)
    clf_if.fit(benign_fuse_train_samples)
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = y_pred

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

100%|██████████| 30/30 [03:09<00:00,  6.32s/it]

{'n_estimators': 10, 'con': 0.2}
0.5290218345806262
              precision    recall  f1-score   support

           0     0.8833    0.7958    0.8373    209540
           1     0.1767    0.2942    0.2208     31212

    accuracy                         0.7307    240752
   macro avg     0.5300    0.5450    0.5290    240752
weighted avg     0.7917    0.7307    0.7573    240752



In [45]:
n_est = [5,6,7,9,10]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    try:
        clf_if = CBLOF(n_clusters=n_est, contamination=con)
        clf_if.fit(normal_fuse_train_samples)
    except:
        continue
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = y_pred

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

100%|██████████| 30/30 [02:45<00:00,  5.52s/it]

{'n_estimators': 10, 'con': 0.1}
0.47841358404676293
              precision    recall  f1-score   support

           0     0.8656    0.8915    0.8783    209540
           1     0.0884    0.0706    0.0785     31212

    accuracy                         0.7850    240752
   macro avg     0.4770    0.4810    0.4784    240752
weighted avg     0.7648    0.7850    0.7746    240752



In [ ]:
# HBOS  Embeddings+Raw (Multimodal)

In [ ]:
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]

In [ ]:
from pyod.models.hbos import HBOS

n_est = [5,10,15,20,25,30]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    clf_if = HBOS(n_bins=n_est, contamination=con)
    clf_if.fit(benign_fuse_train_samples)
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = y_pred

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

  0%|          | 0/36 [00:00<?, ?it/s]

100%|██████████| 36/36 [05:21<00:00,  8.92s/it]

{'n_estimators': 5, 'con': 0.001}
0.37846815552785684
              precision    recall  f1-score   support

           0     0.8219    0.6560    0.7296    698471
           1     0.0195    0.0458    0.0273    104036

    accuracy                         0.5769    802507
   macro avg     0.4207    0.3509    0.3785    802507
weighted avg     0.7179    0.5769    0.6386    802507



In [ ]:
n_est = [5,10,15,20,25,30]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    clf_if = HBOS(n_bins=n_est, contamination=con)
    clf_if.fit(normal_fuse_train_samples)
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = y_pred

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

100%|██████████| 36/36 [05:39<00:00,  9.43s/it]

{'n_estimators': 5, 'con': 0.001}
0.3578868148761849
              precision    recall  f1-score   support

           0     0.8092    0.6025    0.6907    698471
           1     0.0171    0.0465    0.0250    104036

    accuracy                         0.5304    802507
   macro avg     0.4132    0.3245    0.3579    802507
weighted avg     0.7066    0.5304    0.6044    802507



In [ ]:
##  PCA  Emb+Raw (Multimodal/Fusion) Learning

In [ ]:
from pyod.models.pca import PCA
n_est = [5,10,15,20,25,30]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = PCA(n_components=n_est, contamination=con)
    clf_if.fit(benign_fuse_train_samples)
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = y_pred

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

  0%|          | 0/36 [00:00<?, ?it/s]

100%|██████████| 36/36 [03:59<00:00,  6.67s/it]

{'n_estimators': 10, 'con': 0.001}
0.3606674228508121
              precision    recall  f1-score   support

           0     0.8287    0.4315    0.5675    698471
           1     0.0951    0.4012    0.1538    104036

    accuracy                         0.4276    802507
   macro avg     0.4619    0.4164    0.3607    802507
weighted avg     0.7336    0.4276    0.5139    802507



In [ ]:
n_est = [5,10,15,20,25,30]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = PCA(n_components=n_est, contamination=con)
    clf_if.fit(normal_fuse_train_samples)
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = y_pred

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

100%|██████████| 36/36 [04:04<00:00,  6.80s/it]

{'n_estimators': 10, 'con': 0.1}
0.35221219358003625
              precision    recall  f1-score   support

           0     0.8562    0.3553    0.5022    698471
           1     0.1217    0.5995    0.2023    104036

    accuracy                         0.3869    802507
   macro avg     0.4889    0.4774    0.3522    802507
weighted avg     0.7610    0.3869    0.4633    802507



In [ ]:
##  IF  Emb+Raw

In [ ]:
for i in range(len(benign_fuse_train_samples.columns)):
    benign_fuse_train_samples.rename(columns={benign_fuse_train_samples.columns[i]: f"feature {i}"}, inplace=True)

for i in range(len(normal_fuse_train_samples.columns)):
    normal_fuse_train_samples.rename(columns={normal_fuse_train_samples.columns[i]: f"feature {i}"}, inplace=True)

for i in range(len(fuse_test_samples.columns)):
    fuse_test_samples.rename(columns={fuse_test_samples.columns[i]: f"feature {i}"}, inplace=True)

In [ ]:
from sklearn.ensemble import IsolationForest
n_est = [20, 50, 100, 150]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = IsolationForest(n_estimators=n_est, contamination=con)
    clf_if.fit(benign_fuse_train_samples.to_numpy())
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = list(map(lambda x : 0 if x == 1 else 1, y_pred))

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

  0%|          | 0/24 [00:00<?, ?it/s]/home/kienho/miniforge3/envs/nlp-env/lib/python3.9/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but IsolationForest was fitted without feature names
  warnings.warn(
  4%|▍         | 1/24 [00:02<00:49,  2.13s/it]/home/kienho/miniforge3/envs/nlp-env/lib/python3.9/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but IsolationForest was fitted without feature names
  warnings.warn(
  8%|▊         | 2/24 [00:04<00:46,  2.14s/it]/home/kienho/miniforge3/envs/nlp-env/lib/python3.9/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but IsolationForest was fitted without feature names
  warnings.warn(
 12%|█▎        | 3/24 [00:06<00:44,  2.13s/it]/home/kienho/miniforge3/envs/nlp-env/lib/python3.9/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but IsolationForest was fitted without feature names
  warnings.warn(
 17%|█▋        | 4/24 [00:08<00:42,  2.13s/it]/home/kienho/minif

{'n_estimators': 20, 'con': 0.001}
0.46531686753346685
              precision    recall  f1-score   support

           0     0.8701    0.9975    0.9295    698471
           1     0.0338    0.0006    0.0012    104036

    accuracy                         0.8683    802507
   macro avg     0.4520    0.4990    0.4653    802507
weighted avg     0.7617    0.8683    0.8091    802507



In [ ]:
from sklearn.ensemble import IsolationForest

n_est = [20, 50, 100, 150]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = IsolationForest(n_estimators=n_est, contamination=con)
    clf_if.fit(normal_fuse_train_samples)
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = list(map(lambda x : 0 if x == 1 else 1, y_pred))

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

100%|██████████| 24/24 [01:26<00:00,  3.61s/it]

{'n_estimators': 50, 'con': 0.001}
0.46537751706439784
              precision    recall  f1-score   support

           0     0.8703    0.9998    0.9306    698471
           1     0.0545    0.0001    0.0002    104036

    accuracy                         0.8702    802507
   macro avg     0.4624    0.4999    0.4654    802507
weighted avg     0.7646    0.8702    0.8100    802507

